In [257]:
import sqlite3
import pandas as pd

In [258]:
conn = sqlite3.connect('teater.db')

In [259]:
cursor = conn.cursor()

In [260]:
# Teatersal
cursor.execute('''
    CREATE TABLE Teatersal (
        Navn VARCHAR(255) PRIMARY KEY NOT NULL,
        Kapasitet INT
    );
''')

# Theaterstykke
cursor.execute('''
    CREATE TABLE Theaterstykke (
        Navn VARCHAR(255) PRIMARY KEY NOT NULL
    );
''')

# Oppgave
cursor.execute('''
    CREATE TABLE Oppgave (
        Beskrivelse TEXT PRIMARY KEY NOT NULL
    );
''')

# Ansatt
cursor.execute('''
    CREATE TABLE Ansatt (
        ansattId INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        epost VARCHAR(255),
        navn VARCHAR(255),
        ansattstatus VARCHAR(255),
        arbeidstittel VARCHAR(255)
    );
''')

# Kundeprofil
cursor.execute('''
    CREATE TABLE Kundeprofil (
        Mobilnummer VARCHAR(20) PRIMARY KEY NOT NULL,
        Navn VARCHAR(255),
        Adresse TEXT
    );
''')

# Sesong
cursor.execute('''
    CREATE TABLE Sesong (
        sesongId INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        navn VARCHAR(255),
        startdato DATE,
        sluttdato DATE
    );
''')

# Stol
cursor.execute('''
    CREATE TABLE Stol (
        RadNr INT,
        StolNr INT,
        Område VARCHAR(255),
        Navn VARCHAR(255) NOT NULL,
        PRIMARY KEY (RadNr, StolNr, Område, Navn),
        FOREIGN KEY (Navn) REFERENCES Teatersal(Navn)
    );
''')

# Billett
cursor.execute('''
    CREATE TABLE Billett (
        RadNr INT,
        StolNr INT,
        Område VARCHAR(255),
        salNavn VARCHAR(255),
        Mobilnummer VARCHAR(20) NOT NULL,
        ForestillingId INTEGER,
        BillettKjopNummer INT,
        Billettype VARCHAR(255),
        TeaterstykkeNavn VARCHAR(255),
        PRIMARY KEY (RadNr, StolNr, Område, Mobilnummer, ForestillingId, BillettKjopNummer, Billettype, TeaterstykkeNavn),
        FOREIGN KEY (Mobilnummer, BillettKjopNummer) REFERENCES Billettkjøp(Mobilnummer, Nummer),
        FOREIGN KEY (Billettype, TeaterstykkeNavn) REFERENCES BilletType(Billettype, TeaterstykkeNavn),
        FOREIGN KEY (RadNr, StolNr, Område, salNavn) REFERENCES Stol(RadNr, StolNr, Område, Navn),
        FOREIGN Key (ForestillingId) REFERENCES Forestilling(id)
    );
''')

# Billettkjøp
cursor.execute('''
    CREATE TABLE Billettkjøp (
        Nummer INT NOT NULL,
        Mobilnummer VARCHAR(20) NOT NULL,
        dato DATE,
        tid TIME,
        PRIMARY KEY (Nummer, Mobilnummer),
        FOREIGN KEY (Mobilnummer) REFERENCES Kundeprofil(Mobilnummer)
    );
''')

# Forestilling
cursor.execute('''
    CREATE TABLE Forestilling (
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        teaterstykkeNavn VARCHAR(255),
        dato DATE,
        tid TIME,
        FOREIGN KEY (teaterstykkeNavn) REFERENCES Theaterstykke(Navn)
    );
''')

# BilletType
cursor.execute('''
    CREATE TABLE BilletType (
        teaterstykkeNavn VARCHAR(255) NOT NULL,
        type VARCHAR(255) NOT NULL,
        pris DECIMAL,
        minAntall INT,
        PRIMARY KEY (teaterstykkeNavn, type),
        FOREIGN KEY (teaterstykkeNavn) REFERENCES Theaterstykke(Navn)
    );
''')

# Rolle
cursor.execute('''
    CREATE TABLE Rolle (
        teaterstykkeNavn VARCHAR(255) NOT NULL,
        navn VARCHAR(255) NOT NULL,
        PRIMARY KEY (teaterstykkeNavn, navn),
        FOREIGN KEY (teaterstykkeNavn) REFERENCES Theaterstykke(Navn)
    );
''')

# Akt
cursor.execute('''
    CREATE TABLE Akt (
        teaterstykkeNavn VARCHAR(255) NOT NULL,
        nummer INT NOT NULL,
        aktnavn VARCHAR(255),
        PRIMARY KEY (teaterstykkeNavn, nummer),
        FOREIGN KEY (teaterstykkeNavn) REFERENCES Theaterstykke(Navn)
    );
''')

# Relation tables

# RollePaAkt
cursor.execute('''
    CREATE TABLE RollePaAkt (
        teaterstykkeNavn VARCHAR(255) NOT NULL,
        Actnummer INT NOT NULL,
        rolleNavn VARCHAR(255) NOT NULL,
        PRIMARY KEY (teaterstykkeNavn, Actnummer, rolleNavn),
        FOREIGN KEY (teaterstykkeNavn) REFERENCES Theaterstykke(Navn),
        FOREIGN KEY (rolleNavn) REFERENCES Rolle(navn)
    );
''')

# Task
cursor.execute('''
    CREATE TABLE Task (
        Beskrivelse TEXT NOT NULL,
        Identifikator INT NOT NULL,
        Navn VARCHAR(255) NOT NULL,
        PRIMARY KEY (Beskrivelse, Identifikator, Navn),
        FOREIGN KEY (Beskrivelse) REFERENCES Oppgave(Beskrivelse),
        FOREIGN KEY (Identifikator) REFERENCES Ansatt(ansattId),
        FOREIGN KEY (Navn) REFERENCES Theaterstykke(Navn)
    );
''')

# Skuespiller
cursor.execute('''
    CREATE TABLE Skuespiller (
        Identifikator INT NOT NULL,
        TeaterstykkeNavn VARCHAR(255) NOT NULL,
        RolleNavn VARCHAR(255) NOT NULL,
        PRIMARY KEY (Identifikator, TeaterstykkeNavn, RolleNavn),
        FOREIGN KEY (Identifikator) REFERENCES Ansatt(ansattId),
        FOREIGN KEY (TeaterstykkeNavn) REFERENCES Theaterstykke(Navn),
        FOREIGN KEY (RolleNavn) REFERENCES Rolle(navn)
    );
''')


# Display the list of tables created in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
tables


[('Teatersal',),
 ('Theaterstykke',),
 ('Oppgave',),
 ('Ansatt',),
 ('sqlite_sequence',),
 ('Kundeprofil',),
 ('Sesong',),
 ('Stol',),
 ('Billett',),
 ('Billettkjøp',),
 ('Forestilling',),
 ('BilletType',),
 ('Rolle',),
 ('Akt',),
 ('RollePaAkt',),
 ('Task',),
 ('Skuespiller',)]

In [261]:
cursor.execute("INSERT INTO Theaterstykke (Navn) VALUES ('Kongsemnene')")
cursor.execute("INSERT INTO Theaterstykke (Navn) VALUES ('Størst av alt er kjærligheten')")

cursor.execute("INSERT INTO Teatersal (Navn, Kapasitet) VALUES ('Hovedscenen', 524)")
cursor.execute("INSERT INTO Teatersal (Navn, Kapasitet) VALUES ('Gamle scene', 332)")

# Insert into Forestilling for 'Kongsemnene'
dates_kongsemnene = ['2024-02-01', '2024-02-02', '2024-02-03', '2024-02-05', '2024-02-06']
for date in (dates_kongsemnene):
    cursor.execute("INSERT INTO Forestilling ( teaterstykkeNavn, dato, tid) VALUES ('Kongsemnene', ?, '19:00')", ( date,))

# Insert into Forestilling for 'Størst av alt er kjærligheten'
dates_stor = ['2024-02-03', '2024-02-06', '2024-02-07', '2024-02-12', '2024-02-13', '2024-02-14']
for date in dates_stor:
    cursor.execute("INSERT INTO Forestilling (teaterstykkeNavn, dato, tid) VALUES ('Størst av alt er kjærligheten', ?, '18:30')", (date,))

# Insert into BilletTyper for 'Kongsemnene'
BilletTyper_kongsemnene = [
    ('Kongsemnene', 'Ordinær', 450, 0),
    ('Kongsemnene', 'Honnør', 380, 0),
    ('Kongsemnene', 'Student', 280, 0),
    ('Kongsemnene', 'Gruppe 10', 420, 10),
    ('Kongsemnene', 'Gruppe honnør 10', 360, 10)
]
cursor.executemany("INSERT INTO BilletType (teaterstykkeNavn, type, pris, minAntall) VALUES (?, ?, ?, ?)", BilletTyper_kongsemnene)

# Insert into BilletTyper for 'Størst av alt er kjærligheten'
BilletTyper_stor = [
    ('Størst av alt er kjærligheten', 'Ordinær', 350, 0),
    ('Størst av alt er kjærligheten', 'Honnør', 300, 0),
    ('Størst av alt er kjærligheten', 'Student', 220, 0),
    ('Størst av alt er kjærligheten', 'Barn', 220, 0),
    ('Størst av alt er kjærligheten', 'Gruppe 10', 320, 10),
    ('Størst av alt er kjærligheten', 'Gruppe honnør 10', 270, 10)
]
cursor.executemany("INSERT INTO BilletType (teaterstykkeNavn, type, pris, minAntall) VALUES (?, ?, ?, ?)", BilletTyper_stor)

# Commit the changes
conn.commit()

In [262]:
# Check the insertion
cursor.execute("SELECT * FROM Theaterstykke")
theaterstykke_data = cursor.fetchall()
cursor.execute("SELECT * FROM Teatersal")
teatersal_data = cursor.fetchall()
cursor.execute("SELECT * FROM Forestilling")
forestilling_data = cursor.fetchall()
cursor.execute("SELECT * FROM BilletType")
BilletTyper_data = cursor.fetchall()

(theaterstykke_data, teatersal_data, forestilling_data, billetttyper_data)


([('Kongsemnene',), ('Størst av alt er kjærligheten',)],
 [('Hovedscenen', 524), ('Gamle scene', 332)],
 [(1, 'Kongsemnene', '2024-02-01', '19:00'),
  (2, 'Kongsemnene', '2024-02-02', '19:00'),
  (3, 'Kongsemnene', '2024-02-03', '19:00'),
  (4, 'Kongsemnene', '2024-02-05', '19:00'),
  (5, 'Kongsemnene', '2024-02-06', '19:00'),
  (6, 'Størst av alt er kjærligheten', '2024-02-03', '18:30'),
  (7, 'Størst av alt er kjærligheten', '2024-02-06', '18:30'),
  (8, 'Størst av alt er kjærligheten', '2024-02-07', '18:30'),
  (9, 'Størst av alt er kjærligheten', '2024-02-12', '18:30'),
  (10, 'Størst av alt er kjærligheten', '2024-02-13', '18:30'),
  (11, 'Størst av alt er kjærligheten', '2024-02-14', '18:30')],
 [('Kongsemnene', 'Ordinær', 450, 0),
  ('Kongsemnene', 'Honnør', 380, 0),
  ('Kongsemnene', 'Student', 280, 0),
  ('Kongsemnene', 'Gruppe 10', 420, 10),
  ('Kongsemnene', 'Gruppe honnør 10', 360, 10),
  ('Størst av alt er kjærligheten', 'Ordinær', 350, 0),
  ('Størst av alt er kjærlighete

In [263]:
main_area_seats = 504
gallery_area_seats = 520 - 504  # From 504 to 520

# Define the number of seats per row in the main area and the number of rows with 24 seats.
seats_per_row_main = 28
rows_with_24_seats = 2

# Calculate the number of full 28-seat rows
full_rows =18

# Insert seats for the main area
for row in range(1, full_rows + 1):
    for seat_number in range(1, seats_per_row_main + 1):
        seat_number = seat_number + (row-1)*28
        if( 466 < seat_number < 471 ): 
            continue
        if ( 494 < seat_number < 499 ): 
            continue
        cursor.execute("INSERT INTO Stol (RadNr, StolNr, Område, Navn) VALUES (?, ?, 'Hovedområde', 'Hovedscenen')",
                       (row, seat_number))

# Insert seats for the gallery area, which has no rows, so we'll consider it as one row with 16 seats.
for seat_number in range(main_area_seats + 1, gallery_area_seats + main_area_seats + 1):
    if( 504 < seat_number < 515): 
        radnr = 1
    else :
        radnr = 2
    cursor.execute("INSERT INTO Stol (RadNr, StolNr, Område, Navn) VALUES (?, ?, 'Galleri', 'Hovedscenen')",
                   (radnr, seat_number))

# Commit the changes to the database
conn.commit()

In [264]:
# Now let's fetch some of the data to verify the insertions

query = "SELECT * FROM Stol where Navn='Hovedscenen' ORDER BY StolNr, Område, RadNr "

cursor.execute(query)
rows = cursor.fetchall()

# Printing the result
print("Seats in the database:")
print("RadNr | StolNr | Område | Teatersal")
for row in rows:
    radnr, stolnr, område, navn = row
    print(f"{radnr or 'None':5} | {stolnr:6} | {område:10} | {navn}")


Seats in the database:
RadNr | StolNr | Område | Teatersal
    1 |      1 | Hovedområde | Hovedscenen
    1 |      2 | Hovedområde | Hovedscenen
    1 |      3 | Hovedområde | Hovedscenen
    1 |      4 | Hovedområde | Hovedscenen
    1 |      5 | Hovedområde | Hovedscenen
    1 |      6 | Hovedområde | Hovedscenen
    1 |      7 | Hovedområde | Hovedscenen
    1 |      8 | Hovedområde | Hovedscenen
    1 |      9 | Hovedområde | Hovedscenen
    1 |     10 | Hovedområde | Hovedscenen
    1 |     11 | Hovedområde | Hovedscenen
    1 |     12 | Hovedområde | Hovedscenen
    1 |     13 | Hovedområde | Hovedscenen
    1 |     14 | Hovedområde | Hovedscenen
    1 |     15 | Hovedområde | Hovedscenen
    1 |     16 | Hovedområde | Hovedscenen
    1 |     17 | Hovedområde | Hovedscenen
    1 |     18 | Hovedområde | Hovedscenen
    1 |     19 | Hovedområde | Hovedscenen
    1 |     20 | Hovedområde | Hovedscenen
    1 |     21 | Hovedområde | Hovedscenen
    1 |     22 | Hovedområde | Hovedsc

In [265]:
# Insert seats for the Galleri area of Gamle scene
galleri_seat_counts = [33, 18, 17]
for row, seats in enumerate(galleri_seat_counts, start=1):
    for seat_number in range(1, seats + 1):
        cursor.execute("INSERT INTO Stol (RadNr, StolNr, Område, Navn) VALUES (?, ?, 'Galleri', 'Gamle scene')",
                       (row, seat_number))

# Insert seats for the Balkong area of Gamle scene
balkong_seat_counts = [18, 27, 22, 17]
for row, seats in enumerate(balkong_seat_counts, start=1):
    for seat_number in range(1, seats + 1):
        cursor.execute("INSERT INTO Stol (RadNr, StolNr, Område, Navn) VALUES (?, ?, 'Balkong', 'Gamle scene')",
                       (row, seat_number))

# Insert seats for the Parket area of Gamle scene
parket_seat_counts = [18, 16, 17, 18, 18, 17, 18, 17, 17, 14]
for row, seats in enumerate(parket_seat_counts, start=1):
    for seat_number in range(1, seats + 1):
        cursor.execute("INSERT INTO Stol (RadNr, StolNr, Område, Navn) VALUES (?, ?, 'Parket', 'Gamle scene')",
                       (row, seat_number))

# Commit the changes to the database
conn.commit()


In [266]:

query = "SELECT * FROM Stol where Navn='Gamle scene' ORDER BY Navn, Område, RadNr, StolNr "

cursor.execute(query)
rows = cursor.fetchall()

# Printing the result
print("Seats in the database:")
print("RadNr | StolNr | Område | Teatersal")
for row in rows:
    radnr, stolnr, område, navn = row
    print(f"{radnr or 'None':5} | {stolnr:6} | {område:10} | {navn}")



Seats in the database:
RadNr | StolNr | Område | Teatersal
    1 |      1 | Balkong    | Gamle scene
    1 |      2 | Balkong    | Gamle scene
    1 |      3 | Balkong    | Gamle scene
    1 |      4 | Balkong    | Gamle scene
    1 |      5 | Balkong    | Gamle scene
    1 |      6 | Balkong    | Gamle scene
    1 |      7 | Balkong    | Gamle scene
    1 |      8 | Balkong    | Gamle scene
    1 |      9 | Balkong    | Gamle scene
    1 |     10 | Balkong    | Gamle scene
    1 |     11 | Balkong    | Gamle scene
    1 |     12 | Balkong    | Gamle scene
    1 |     13 | Balkong    | Gamle scene
    1 |     14 | Balkong    | Gamle scene
    1 |     15 | Balkong    | Gamle scene
    1 |     16 | Balkong    | Gamle scene
    1 |     17 | Balkong    | Gamle scene
    1 |     18 | Balkong    | Gamle scene
    2 |      1 | Balkong    | Gamle scene
    2 |      2 | Balkong    | Gamle scene
    2 |      3 | Balkong    | Gamle scene
    2 |      4 | Balkong    | Gamle scene
    2 |      5 | 

In [267]:
# 1. Create a customer profile in 'Kundeprofil'
customer_profile = ("1234567890", "Kari Nordmann", "Bakkegata 3, 1234 Storbyen")
cursor.execute("INSERT INTO Kundeprofil (Mobilnummer, Navn, Adresse) VALUES (?, ?, ?)", customer_profile)

# 2. Create a record in 'Billettkjøp'
ticket_purchase = (1, "1234567890", "2024-02-03", "18:30")
cursor.execute("INSERT INTO Billettkjøp (Nummer, Mobilnummer, dato, tid) VALUES (?, ?, ?, ?)", ticket_purchase)
conn.commit()

In [268]:
# 3. Buy specified seats
# Define the seating layout as provided, '1' for purchased, '0' for not purchased
seating_layout = """
00000000000000000
000000000000000000
000001100000000000000000000000000
Galleri
00000000000000000
0000000000000000000000
110000000000000000000000000
0000000000000000000000000000
Balkong
00000001000000
00000001000000000
00000001000000000
000000011000000000
00000001000000000
000000011000000000
000110011000000000
00011000100000000
1110000010000000
110000011000000000
Parkett
"""
# Process the seating layout
rows = seating_layout.strip().split("\n")

# reverse the rows 
rows = rows[::-1]
current_area = ""
for row in rows:
    if row in ["Galleri", "Balkong", "Parkett"]:
        current_area = row
        row_counter = 1
    else:
        for seat_counter, seat in enumerate(row, start=1):
            if seat == "1":
                
                cursor.execute("INSERT INTO Billett (RadNr, StolNr, Område, SalNavn, Mobilnummer, ForestillingId, BillettKjopNummer, Billettype, TeaterstykkeNavn ) VALUES (?, ?, ?, 'Gamle scene', ?, ? ,?, ? , ?)", 
                               (row_counter, seat_counter, current_area, "1234567890", 1, 1, "Ordinær", "Størst av alt er kjærligheten"))
        row_counter += 1

# Commit the changes and close the connection
conn.commit()

In [269]:

query = "SELECT RadNr, StolNr, Område, SalNavn FROM Billett ORDER BY SalNavn, Område, RadNr DESC, StolNr DESC"

cursor.execute(query)
bought_seats = cursor.fetchall()

# Printing the result
print("Bought Seats:")
print("RadNr | StolNr | Område       | Teatersal")
for seat in bought_seats:
    radnr, stolnr, område, navn = seat
    print(f"{radnr or 'None':5} | {stolnr:6} | {område:12} | {navn}")



Bought Seats:
RadNr | StolNr | Område       | Teatersal
    2 |      2 | Balkong      | Gamle scene
    2 |      1 | Balkong      | Gamle scene
    1 |      7 | Galleri      | Gamle scene
    1 |      6 | Galleri      | Gamle scene
   10 |      8 | Parkett      | Gamle scene
    9 |      8 | Parkett      | Gamle scene
    8 |      8 | Parkett      | Gamle scene
    7 |      9 | Parkett      | Gamle scene
    7 |      8 | Parkett      | Gamle scene
    6 |      8 | Parkett      | Gamle scene
    5 |      9 | Parkett      | Gamle scene
    5 |      8 | Parkett      | Gamle scene
    4 |      9 | Parkett      | Gamle scene
    4 |      8 | Parkett      | Gamle scene
    4 |      5 | Parkett      | Gamle scene
    4 |      4 | Parkett      | Gamle scene
    3 |      9 | Parkett      | Gamle scene
    3 |      5 | Parkett      | Gamle scene
    3 |      4 | Parkett      | Gamle scene
    2 |      9 | Parkett      | Gamle scene
    2 |      3 | Parkett      | Gamle scene
    2 |      2 | Par

In [270]:



# Define the seating pattern for the tickets to be purchased
seating_pattern = [
    "000000000000000000xxxx000000",
    "000000000000000000xxxx000000",
    "0000000000100000000000000000",
    "0000011100100000000000000000",
    "0000000000100000000000000000",
    "1110000000100000000000000000",
    "0000000000000000000000000000",
    "0000000000000000000000000000",
    "0000000000000000000000000000",
    "0000000000000000000000000000",
    "0000000000000000000000000000",
    "0000000000001100000000000000",
    "0000000000000110000000000000",
    "0000000000000011000000000000",
    "0000000000000001100000000000",
    "0000000001111111111000000000",
    "1111111111111111111000000000",
    "0000000000111111111111111111"
]

# Identify the seats to be purchased
def find_seats_to_purchase(seating_pattern):
    seats_to_purchase = []
    
    seating_pattern = seating_pattern[::-1]
    for row_number, row in enumerate(seating_pattern, start=1):
        for seat_number, seat in enumerate(row, start=1):
            seatnumber_sum = seat_number + (row_number-1)*28
            if seat == "1":
                seats_to_purchase.append((row_number, seatnumber_sum))
    return seats_to_purchase

# Seats to be purchased
seats_to_purchase = find_seats_to_purchase(seating_pattern)
print(seats_to_purchase)

performance_date = "2024-02-03"
get_forestillingId = "SELECT id FROM Forestilling WHERE teaterstykkeNavn = 'Kongsemnene' AND dato = ?"
cursor.execute(get_forestillingId, (performance_date,))
forestilling_id = cursor.fetchone()[0]

print(forestilling_id)


# Insert tickets for each selected seat
customer_mobile_number = "1234567890"  # Assuming this is the customer's mobile number
forestilling_id = 1  # Assuming an ID for the 'Forestilling'. Replace with the correct ID.

for seat in seats_to_purchase:
    radnr, stolnr = seat
    omrade = "Hovedområde" if radnr <= len(seating_pattern) else "Galleri"
    sal_navn = "Hovedscenen"  
    billettype = "Ordinær"  

    # Prepare the SQL command
    sql_command = """
        INSERT INTO Billett 
        (RadNr, StolNr, Område, salNavn, Mobilnummer, ForestillingId, BillettKjopNummer, Billettype, TeaterstykkeNavn)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """
    values = (radnr, stolnr, omrade, sal_navn, customer_mobile_number, forestilling_id, 1, billettype, 'Kongsemnene')

    # Execute the command
    cursor.execute(sql_command, values)

# # Commit the changes
conn.commit()


[(1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (2, 29), (2, 30), (2, 31), (2, 32), (2, 33), (2, 34), (2, 35), (2, 36), (2, 37), (2, 38), (2, 39), (2, 40), (2, 41), (2, 42), (2, 43), (2, 44), (2, 45), (2, 46), (2, 47), (3, 66), (3, 67), (3, 68), (3, 69), (3, 70), (3, 71), (3, 72), (3, 73), (3, 74), (3, 75), (4, 100), (4, 101), (5, 127), (5, 128), (6, 154), (6, 155), (7, 181), (7, 182), (13, 337), (13, 338), (13, 339), (13, 347), (14, 375), (15, 398), (15, 399), (15, 400), (15, 403), (16, 431)]
3


In [271]:
query = "SELECT RadNr, StolNr, Område, SalNavn FROM Billett where TeaterstykkeNavn = 'Kongsemnene' ORDER BY SalNavn, Område, RadNr DESC, StolNr DESC"

cursor.execute(query)
bought_seats = cursor.fetchall()

# Printing the result
print("Bought Seats:")
print("RadNr | StolNr | Område       | Teatersal")
for seat in bought_seats:
    radnr, stolnr, område, navn = seat
    print(f"{radnr or 'None':5} | {stolnr:6} | {område:12} | {navn}")

Bought Seats:
RadNr | StolNr | Område       | Teatersal
   16 |    431 | Hovedområde  | Hovedscenen
   15 |    403 | Hovedområde  | Hovedscenen
   15 |    400 | Hovedområde  | Hovedscenen
   15 |    399 | Hovedområde  | Hovedscenen
   15 |    398 | Hovedområde  | Hovedscenen
   14 |    375 | Hovedområde  | Hovedscenen
   13 |    347 | Hovedområde  | Hovedscenen
   13 |    339 | Hovedområde  | Hovedscenen
   13 |    338 | Hovedområde  | Hovedscenen
   13 |    337 | Hovedområde  | Hovedscenen
    7 |    182 | Hovedområde  | Hovedscenen
    7 |    181 | Hovedområde  | Hovedscenen
    6 |    155 | Hovedområde  | Hovedscenen
    6 |    154 | Hovedområde  | Hovedscenen
    5 |    128 | Hovedområde  | Hovedscenen
    5 |    127 | Hovedområde  | Hovedscenen
    4 |    101 | Hovedområde  | Hovedscenen
    4 |    100 | Hovedområde  | Hovedscenen
    3 |     75 | Hovedområde  | Hovedscenen
    3 |     74 | Hovedområde  | Hovedscenen
    3 |     73 | Hovedområde  | Hovedscenen
    3 |     72 | Hov